In [27]:
import os
import stata_setup
## 设置pystata目录
stata_setup.config(os.getenv("STATA_SYSDIR"), 'mp')
from pystata import stata

下面以数据集${nomocc2.dta}$为例, 进行多项 Logit 与多项 Probit 估计。此数据集来自 1982 年美国 General Social Survey, 该问卷调查将受访者职业分为五类 (${occ}$), 即服务人员 (${menial}$), 蓝领 (${blue collar}$), 工匠 (${craft}$), 白领 (${white collar}$) 以及专业人士 (${professional}$)。解释变量包括: 是否白人 (${white}$), 受教育年限 (${ed}$) 以及工龄 (${exper}$)。显然, 这三个解释变量都只依赖于个体, 而不依赖于方案, 故使用多项 Logit 或多项 Probit 回归。

In [11]:
stata.run('use nomocc2.dta, replace')
data = stata.pdataframe_from_data()
data.head()

(1982 General Social Survey)


,occ,white,ed,exper
0,1,1,11,3
1,1,1,12,14
2,1,1,12,44
3,1,1,12,18
4,1,0,14,24


先看一下数据的基本特征

In [12]:
stata.run('sum')


    Variable |        Obs        Mean    Std. dev.       Min        Max
-------------+---------------------------------------------------------
         occ |        337    3.397626    1.367913          1          5
       white |        337    .9169139    .2764227          0          1
          ed |        337    13.09496    2.946427          3         20
       exper |        337    20.50148    13.95936          2         66


由于数据按职业排序，故前6名都从事服务业。其中，5名为白人，受教育年限从11-14年不等，而工龄从3-44年不等。由于数据表中每行对应于一名个体，故数据格式为宽形(wide form)。

下面通过列表粗略地考察受教育年限(ed)与职业(occ)的关系：
<br>tabstat ed, by(occ) statistics(N mean sd)<br>
其中，命令“table”表示将变量的统计特征列表(tables of summary statistics)，而选择项“contents()”则用来指定最多5个统计量，“N ed mean ed sd ed”表示罗列变量ed的样本容量、均值与标准差（按变量occ分为5个子样本）。


In [15]:
stata.run('tabstat ed, by(occ) statistics(N mean sd)')


Summary for variables: ed
Group variable: occ (Occupation)

     occ |         N      Mean        SD
---------+------------------------------
  Menial |        31  11.77419  2.186469
 BlueCol |        69  11.21739  2.571733
   Craft |        84  11.96429  2.119595
WhiteCol |        41  13.17073  2.096455
    Prof |       112   15.4375  2.608998
---------+------------------------------
   Total |       337  13.09496  2.946427
----------------------------------------


从上表可知，在样本中专业人士最多，且平均受教育年限最高；而蓝领的受教育程度最低。下面进行多项 Logit 回归

In [16]:
stata.run('mlogit occ white ed exper, nolog')


Multinomial logistic regression                         Number of obs =    337
                                                        LR chi2(12)   = 166.09
                                                        Prob > chi2   = 0.0000
Log likelihood = -426.80048                             Pseudo R2     = 0.1629

------------------------------------------------------------------------------
         occ | Coefficient  Std. err.      z    P>|z|     [95% conf. interval]
-------------+----------------------------------------------------------------
Menial       |
       white |  -1.774306   .7550543    -2.35   0.019    -3.254186   -.2944273
          ed |  -.7788519   .1146293    -6.79   0.000    -1.003521   -.5541826
       exper |  -.0356509    .018037    -1.98   0.048    -.0710028    -.000299
       _cons |   11.51833   1.849356     6.23   0.000     7.893659      15.143
-------------+----------------------------------------------------------------
BlueCol      |
       white |  -.53

由于没有指定参照方案(base outcome)，故命令$mlogit$自动选择观测值最多的方案（即专业人士）为参照方案。上表显示，在 5\% 的显著性水平上，给定其他变量，白人 ($white$) 更不可能选择服务业或工匠；但是否白人对于选择蓝领或白领没有显著影响。受教育程度 ($ed$) 越高，越不可能选择除专业人士以外的职业。工龄越长 ($exper$)，越不可能选择服务业或蓝领；工龄对于选择工匠或白领无显著影响。

由于 IIA 假定是多项 Logit 模型的前提，下面检验 IIA 假定是否满足。


In [32]:
stata.run('mlogtest, hausman base')


Problem determining number of categories.

**** Hausman tests of IIA assumption

 Ho: Odds(Outcome-J vs Outcome-K) are independent of other alternatives.
You used the old syntax of hausman. Click here to learn about the new syntax.

(storing estimation results as _HAUSMAN)

 Omitted |      chi2   df   P>chi2   evidence
---------+------------------------------------
  Menial |     7.324   12    0.835   for Ho    
 BlueCol |     0.320   12    1.000   for Ho    
   Craft |   -14.436   12    1.000   for Ho    
WhiteCol |    -5.541   11    1.000   for Ho    
    Prof |    15.566   12    0.212   for Ho    
----------------------------------------------


上表的前四行豪斯曼检验结果显示，去掉四个非参照方案（nonbase alternatives）中的任何一个方案，都不会拒绝 IIA 的原假设。由于使用了选择项“base”，故上表第五行计算去掉参照方案（Prof），而以剩余方案中观测值最多的方案作为参照方案的检验结果，同样也不拒绝 IIA 假设。但由于某种原因，却无法进行 Small-Hsiao 检验。


上文提到，这两个检验的小样本性质都不好，只具有参考价值；但至少没有发现违背IIA假定的迹象（可能这五类职业太不相同了）。
如果要显示相对风险比率($\mathrm{P}(y=j)/\mathrm{P}(y=1)=\exp(x_i'\beta_j)$)
，可以输入命令

In [33]:
stata.run('mlogit occ white ed exper, nolog rrr')


Multinomial logistic regression                         Number of obs =    337
                                                        LR chi2(12)   = 166.09
                                                        Prob > chi2   = 0.0000
Log likelihood = -426.80048                             Pseudo R2     = 0.1629

------------------------------------------------------------------------------
         occ |        RRR   Std. err.      z    P>|z|     [95% conf. interval]
-------------+----------------------------------------------------------------
Menial       |
       white |    .169601    .128058    -2.35   0.019     .0386123    .7449581
          ed |   .4589326   .0526071    -6.79   0.000     .3665863    .5745417
       exper |   .9649771   .0174053    -1.98   0.048     .9314593     .999701
       _cons |     100542   185937.9     6.23   0.000     2680.234     3771571
-------------+----------------------------------------------------------------
BlueCol      |
       white |   .58

下面，根据模型预测个体选择各种职业的可能性，分别记为 occ1, occ2, occ3, occ4, occ5，并显示对前 5 个观测值的预测结果。

In [36]:
%%stata
predict occ1 occ2 occ3 occ4 occ5
list occ1-occ5 in 1/5


. drop occ1 occ2 occ3 occ4 occ5

. predict occ1 occ2 occ3 occ4 occ5
(option pr assumed; predicted probabilities)

. list occ1-occ5 in 1/5

     +------------------------------------------------------+
     |     occ1       occ2       occ3       occ4       occ5 |
     |------------------------------------------------------|
  1. | .1681295   .4128002   .2760952    .085288   .0576871 |
  2. | .1257816   .2945018   .3076293   .1328948   .1391926 |
  3. | .0644456   .1738508   .3616529   .1922331   .2078175 |
  4. | .1161744   .2771936   .3174044   .1409616    .148266 |
  5. | .1691383   .0988214    .410424   .1063373    .215279 |
     +------------------------------------------------------+

. 


也可以选择其他职业作为参照方案，比如服务业

In [37]:
%stata mlogit occ white ed exper,base(1) nolog


Multinomial logistic regression                         Number of obs =    337
                                                        LR chi2(12)   = 166.09
                                                        Prob > chi2   = 0.0000
Log likelihood = -426.80048                             Pseudo R2     = 0.1629

------------------------------------------------------------------------------
         occ | Coefficient  Std. err.      z    P>|z|     [95% conf. interval]
-------------+----------------------------------------------------------------
Menial       |  (base outcome)
-------------+----------------------------------------------------------------
BlueCol      |
       white |   1.236504   .7244352     1.71   0.088    -.1833631    2.656371
          ed |  -.0994247   .1022812    -0.97   0.331    -.2998922    .1010428
       exper |   .0047212   .0173984     0.27   0.786    -.0293789    .0388214
       _cons |   .7412336    1.51954     0.49   0.626     -2.23701    3.719477
----

从上表可知，系数估计值随参照方案的不同而变化。下面，使用 multinomial probit 来估计此模型

In [39]:
%stata mprobit occ white ed exper, nolog


Multinomial probit regression                           Number of obs =    337
                                                        Wald chi2(12) = 105.61
Log likelihood = -429.31856                             Prob > chi2   = 0.0000

------------------------------------------------------------------------------
         occ | Coefficient  Std. err.      z    P>|z|     [95% conf. interval]
-------------+----------------------------------------------------------------
Menial       |
       white |  -1.144907   .5027501    -2.28   0.023    -2.130279   -.1595352
          ed |  -.5094985   .0698816    -7.29   0.000    -.6464639   -.3725331
       exper |  -.0234636   .0109546    -2.14   0.032    -.0449343   -.0019929
       _cons |    7.46242   1.145854     6.51   0.000     5.216587    9.708253
-------------+----------------------------------------------------------------
BlueCol      |
       white |   -.392222   .5182974    -0.76   0.449    -1.408066    .6236222
          ed |  -.58

从上表可知，多项 Probit 的系数估计值与多项 Logit 并不相同，但二者的系数不具有可比性；具有可比性的是两个模型的预测概率。为此，计算多项 Probit 模型所预测的选择概率，分别记为 occ1p, occ2p, occ3p, occ4p, occ5p

In [41]:
%stata predict occ1p occ2p occ3p occ4p occ5p

(option pr assumed; predicted probabilities)


下面计算两个模型职业预测概率的相关性

In [44]:
%%stata
corr occ1 occ1p
corr occ2 occ2p
corr occ3 occ3p
corr occ4 occ4p
corr occ5 occ5p


. corr occ1 occ1p
(obs=337)

             |     occ1    occ1p
-------------+------------------
        occ1 |   1.0000
       occ1p |   0.9979   1.0000


. corr occ2 occ2p
(obs=337)

             |     occ2    occ2p
-------------+------------------
        occ2 |   1.0000
       occ2p |   0.9985   1.0000


. corr occ3 occ3p
(obs=337)

             |     occ3    occ3p
-------------+------------------
        occ3 |   1.0000
       occ3p |   0.9935   1.0000


. corr occ4 occ4p
(obs=337)

             |     occ4    occ4p
-------------+------------------
        occ4 |   1.0000
       occ4p |   0.9929   1.0000


. corr occ5 occ5p
(obs=337)

             |     occ5    occ5p
-------------+------------------
        occ5 |   1.0000
       occ5p |   0.9989   1.0000


. 


由以上结果可知，两个模型所预测的职业选择概率高度一致，相关系数均在99\%以上。这意味着，使用多项Logit或多项Probit在实际上并无多少区别；只是多项Probit的计算时间更长，且无法从几率比角度解释系数估计值，故实践中常使用多项Logit。

下面以数据集 travel2.dta 为例，进行条件 Logit 与混合 Logit 的估计。该数据集的观测单位为 152 组人群（每组可视为一个小型旅行团），每个人群从以下三种度假旅行方式中选择一种，即火车、长途大巴或自驾车。随方案而变的解释变量包括 time（总旅行时间）与 invc（乘车成本，in-vehicle cost）。不随方案而变的解释变量包括 hinc（家庭收入，household income）与 psize（旅行团人数，party size）。

首先，通过前6个观测值来考察数据格式。

In [46]:
%stata use travel2.dta, clear

(Greene & Hensher 1997 data on travel mode choice)


In [59]:
data = stata.pdataframe_from_data()
data.head()

,id,mode,train,bus,car,time,invc,choice,ttme,invt,gc,hinc,psize,prob
0,1,1,1,0,0,406,31,0,34,372,71,35,1,0.064248
1,1,2,0,1,0,452,25,0,35,417,70,35,1,0.010720
2,1,3,0,0,1,180,10,1,0,180,30,35,1,0.925032
3,2,1,1,0,0,398,31,0,44,354,84,30,2,0.253561
4,2,2,0,1,0,452,25,0,53,399,85,30,2,0.036301


In [49]:
%stata list id mode train bus time invc choice hinc psize in 1/6, sepby(id)


     +----------------------------------------------------------------+
     | id    mode   train   bus   time   invc   choice   hinc   psize |
     |----------------------------------------------------------------|
  1. |  1   Train       1     0    406     31        0     35       1 |
  2. |  1     Bus       0     1    452     25        0     35       1 |
  3. |  1     Car       0     0    180     10        1     35       1 |
     |----------------------------------------------------------------|
  4. |  2   Train       1     0    398     31        0     30       2 |
  5. |  2     Bus       0     1    452     25        0     30       2 |
  6. |  2     Car       0     0    255     11        1     30       2 |
     +----------------------------------------------------------------+


尽管数据集中只有152个旅行团，但由于每个旅行团占据3行数据，故实际的样本容量为456（即152×3）。下面看一下数据的统计特征。

In [50]:
%stata sum id mode train bus time invc choice hinc psize


    Variable |        Obs        Mean    Std. dev.       Min        Max
-------------+---------------------------------------------------------
          id |        456    103.8882    61.03044          1        210
        mode |        456           2    .8173933          1          3
       train |        456    .3333333    .4719223          0          1
         bus |        456    .3333333    .4719223          0          1
        time |        456    632.1096    270.2547        180       1440
-------------+---------------------------------------------------------
        invc |        456    33.95175      21.795          2        109
      choice |        456    .3333333    .4719223          0          1
        hinc |        456    31.80921    19.25813          2         72
       psize |        456    1.809211    1.069457          1          6



下面进行条件 Logit 估计。由于命令 clogit 只接受随方案而变的解释变量，因此仅使用总旅行时间（单位为分钟）、乘车成本、虚拟变量 train 以及虚拟变量 bus 作为解释变量（以旅行方式 car 为参照方案）

In [51]:
%stata clogit choice train bus time invc, group(id) nolog


Conditional (fixed-effects) logistic regression         Number of obs =    456
                                                        LR chi2(4)    = 172.06
                                                        Prob > chi2   = 0.0000
Log likelihood = -80.961135                             Pseudo R2     = 0.5152

------------------------------------------------------------------------------
      choice | Coefficient  Std. err.      z    P>|z|     [95% conf. interval]
-------------+----------------------------------------------------------------
       train |   2.671238   .4531611     5.89   0.000     1.783058    3.559417
         bus |   1.472335   .4007152     3.67   0.000     .6869474    2.257722
        time |  -.0191453   .0024509    -7.81   0.000    -.0239489   -.0143417
        invc |  -.0481658   .0119516    -4.03   0.000    -.0715905   -.0247411
------------------------------------------------------------------------------


上表表明，在其他解释变量（如总旅行时间和乘车成本）保持不变的情况下，旅行团最有可能选择火车作为出行方式，其次是长途大巴。此外，当一个方案的总旅行时间较长且乘车成本较高时，选择该方案的概率会降低。然而，由于这是一个非线性模型，直接通过系数估计值来评估边际效应较为困难。因此，为了计算风险比率，可以在上述命令中添加选项“or”。

In [53]:
%stata clogit choice train bus time invc, group(id) nolog or


Conditional (fixed-effects) logistic regression         Number of obs =    456
                                                        LR chi2(4)    = 172.06
                                                        Prob > chi2   = 0.0000
Log likelihood = -80.961135                             Pseudo R2     = 0.5152

------------------------------------------------------------------------------
      choice | Odds ratio   Std. err.      z    P>|z|     [95% conf. interval]
-------------+----------------------------------------------------------------
       train |   14.45786   6.551738     5.89   0.000      5.94802    35.14272
         bus |   4.359401   1.746879     3.67   0.000     1.987639    9.561286
        time |   .9810368   .0024044    -7.81   0.000     .9763356    .9857607
        invc |   .9529758   .0113896    -4.03   0.000      .930912    .9755624
------------------------------------------------------------------------------


从上表可以看出，变量 time 的风险比为 0.98，这意味着在控制其他变量不变的情况下，当一个方案的总旅行时间每增加1分钟时，选择该方案的概率会减少2%。同样地，变量 invc（乘车成本）的风险比也可以这样解释。此外，虚拟变量 bus 的风险比为 4.36，这表明在所有方案的时间和成本都相同的情况下，旅行团选择长途大巴的可能性是选择自驾车的4.36倍。虚拟变量 train 的风险比也可以用同样的方式来解释

下面计算条件 Logit 模型的预测概率

In [55]:
stata.run('predict prob')

(option pc1 assumed; probability of success given one success within group)


In [56]:
%stata list id mode prob choice time invc in 1/3


     +----------------------------------------------+
     | id    mode       prob   choice   time   invc |
     |----------------------------------------------|
  1. |  1   Train   .0642477        0    406     31 |
  2. |  1     Bus   .0107205        0    452     25 |
  3. |  1     Car   .9250318        1    180     10 |
     +----------------------------------------------+


上表显示，第1个旅行团实际选择自驾车（Car），而模型预测选择自驾车的概率高达0.925，而且正好是旅行时间与成本最低的方案。

对于条件 Logit 模型，也可用命令 asclogit 来估计

In [60]:
%stata asclogit choice time invc, case(id) alternatives(mode) base(3) nolog


Alternative-specific conditional logit         Number of obs      =        456
Case ID variable: id                           Number of cases    =        152

Alternatives variable: mode                    Alts per case: min =          3
                                                              avg =        3.0
                                                              max =          3

                                                  Wald chi2(2)    =      70.53
Log likelihood = -80.961135                       Prob > chi2     =     0.0000

------------------------------------------------------------------------------
      choice | Coefficient  Std. err.      z    P>|z|     [95% conf. interval]
-------------+----------------------------------------------------------------
mode         |
        time |  -.0191453   .0024509    -7.81   0.000    -.0239489   -.0143417
        invc |  -.0481658   .0119516    -4.03   0.000    -.0715905   -.0247411
-------------+-------------------

对比上表与命令 clogit 的估计结果可知，二者的系数估计值与标准误均完全相同。使用命令 asclogit 的主要好处在于，它可以通过选择项“casevars(varname)”将只随个体而变的解释变量也包括在模型中。下面将解释变量“家庭收入”（hinc）与“旅行团人数”（psize）也包括进来，进行混合 Logit 模型估计。

In [61]:
%stata asclogit choice time invc, case(id) alternatives(mode) base(3) casevars(hinc psize) nolog


Alternative-specific conditional logit         Number of obs      =        456
Case ID variable: id                           Number of cases    =        152

Alternatives variable: mode                    Alts per case: min =          3
                                                              avg =        3.0
                                                              max =          3

                                                  Wald chi2(6)    =      69.09
Log likelihood = -77.504846                       Prob > chi2     =     0.0000

------------------------------------------------------------------------------
      choice | Coefficient  Std. err.      z    P>|z|     [95% conf. interval]
-------------+----------------------------------------------------------------
mode         |
        time |  -.0185035   .0025035    -7.39   0.000    -.0234103   -.0135966
        invc |  -.0402791   .0134851    -2.99   0.003    -.0667095   -.0138488
-------------+-------------------

上表显示，家庭收入（hinc）越高，越不倾向于选择火车；但对于选择长途大巴则无显著影响。旅行团规模（psize）没有显著影响。旅行时间（time）与乘车成本（invc）的估计系数依然显著为负，且与前面条件 Logit 模型的估计系数很接近。上表并不汇报准 $R^2$，但可以很容易地手工计算。从上表可知，该模型的对数似然函数值为 -77.504846，下面估计一个只含常数项的模型

In [62]:
%stata asclogit choice, case(id) alternatives(mode) base(3) nolog


Alternative-specific conditional logit         Number of obs      =        456
Case ID variable: id                           Number of cases    =        152

Alternatives variable: mode                    Alts per case: min =          3
                                                              avg =        3.0
                                                              max =          3

                                                  Wald chi2(0)    =          .
Log likelihood = -160.00172                       Prob > chi2     =          .

------------------------------------------------------------------------------
      choice | Coefficient  Std. err.      z    P>|z|     [95% conf. interval]
-------------+----------------------------------------------------------------
Train        |
       _cons |   .0655973   .1811689     0.36   0.717    -.2894872    .4206818
-------------+----------------------------------------------------------------
Bus          |
       _cons |    

上表显示，只含常数项模型的对数似然函数为-160.00172，故可计算准$R^2$如下

In [63]:
%stata dis(160.00172-77.504846)/160.00172

.51559992
